<a href="https://colab.research.google.com/github/dylan-jacobs/quant-trading-algorithms/blob/main/alpaca_trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install alpaca_trade_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.7/757.7 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 11.4 MB/s eta 0:00:00
  Created wheel for msgpack: filename=msgpack-1.0.3-cp311-cp311-linux_x86_64.whl size=15688 sha256=72985764217594127c74c5a294e5978f9a5b9ce97a96b4964311d78c45c2f554
  Stored in directory: /root/.cache/pip/wheels/f6/35/da/ed9b26b510235e00e3a3c3bab7bad97b59214729662255ab3d
Successfully built msgpack
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.1.0
    Uninstalling msgpack-1.1.0:
      Successfully uninstalled msgpack-1.1.0
  Attempting uninstall: websockets
    Found existing installation: websockets 14.1
    Uninstalling

In [8]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 8 2022

@author: dylan

Strategy: semi hft?
Every five mins:
    iterate throught list of tickers (all from s&p500 index)
    find one that is at low bollinger index
    buy
    keep track of bought stocks and profits
        when owned stocks are at too high bollinger -> sell, or when price has risen by $1
        sell when lost > 5% of initial dinero
"""

import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import yfinance
import time
from datetime import datetime
import math
from pytz import timezone
import sys
import requests
from threading import Thread
import stock_predictor

# ALPACA API Info for fetching data, portfolio, etc. from Alpaca
BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_API_KEY = "PKAU1W5OWUM6942S3MM0"
ALPACA_SECRET_KEY = "BPJgvm8F9SiWsrufM5NqtcdY3rlEHchaxOELM0Yy"
TICKER = 'SPY'

# Instantiate REST API Connection
api = tradeapi.REST(key_id=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY,
                    base_url=BASE_URL, api_version='v2')

class NewThread(Thread):
    def __init__(self, group=None, target=None, name=None, args=(), kwargs={}):
        Thread.__init__(self, group, target, name, args, kwargs)

    def run(self):
        if self._target != None:
            self._return = self._target(*self._args, **self._kwargs)

    def join(self, *args):
        Thread.join(self, *args)
        return self._return

def getAlpacaQuote(ticker):
    try:
        quote = api.get_latest_quote(ticker)
    except:
        quote = None
    return quote

def placeBuyAlpacaOrder(ticker, amnt, price, orders, positions):
    return api.submit_order(symbol=ticker, qty=amnt, side='buy', type='limit', time_in_force='day', limit_price=price)

def placeSellAlpacaOrder(ticker, amnt, price, limit_order):
    response = None

    if len(positions) > 0: # we own at least one stock
        if limit_order:
            response = api.submit_order(symbol=ticker, qty=amnt, side='sell', type='limit', time_in_force='day', limit_price=price)
        else:
            response = api.submit_order(symbol=ticker, qty=amnt, side='sell', type='market', time_in_force='day')
    else:
        print('We do not own this stock!')

    return response

def getTime():
    CURRENT_HOUR = int(datetime.now(timezone('US/Eastern')).strftime('%H'))
    CURRENT_MIN = int(datetime.now(timezone('US/Eastern')).strftime('%M'))
    CURRENT_SEC = int(datetime.now(timezone('US/Eastern')).strftime('%S'))
    return CURRENT_HOUR, CURRENT_MIN, CURRENT_SEC

def setAccountVars():
    try:
        account = api.get_account()
        positions = api.list_positions()
        orders = api.list_orders()
        CASH = float(account.cash)
        EQUITY = float(account.equity)
        PROFIT = 0
        if (len(positions) > 0):
            for pos in positions:
                PROFIT += float(pos.unrealized_pl)
        return positions, orders, CASH, EQUITY, PROFIT
    except Exception:
        time.sleep(5)
        return setAccountVars()

def buyPosition(ticker, qty, price):
    positions, orders, CASH, _, _ = setAccountVars()

    # buy now
    response = placeBuyAlpacaOrder(ticker, qty, price, orders, positions)

    # update info
    ORDER_ID = response.id
    positions, orders, CASH, _, _ = setAccountVars()
    times_to_delay = 0
    symbols = [p.symbol for p in positions]
    while (ticker not in symbols) and (times_to_delay <= 24):
        positions, orders, CASH, _, _ = setAccountVars()
        symbols = [p.symbol for p in positions]
        times_to_delay+=1
        print('Attempting to buy...')
        time.sleep(5)
    if (ticker not in symbols):
        api.cancel_order(ORDER_ID)

def sellPosition(ticker, qty, price, limit_order):
    # sell now
    response = placeSellAlpacaOrder(ticker, qty, price, limit_order)
    positions, orders, CASH, EQUITY, PROFIT = setAccountVars()

    # update info
    symbols = [p.symbol for p in positions]
    times_repeated = 0
    while (ticker in symbols) and times_repeated <= 60:
        symbols = [p.symbol for p in api.list_positions()]
        print('Attempting to sell...')
        times_repeated += 1
        time.sleep(5)
    return response

def run():
    # get time
    CURRENT_HOUR, CURRENT_MIN, _ = getTime()

    # is it trading hours?
    if(((CURRENT_HOUR + (CURRENT_MIN / 60)) >= 9.5) & (CURRENT_HOUR < 16) & (datetime.today().weekday() != 5) & (datetime.today().weekday() != 6)) or True:

        # get account info
        positions, orders, CASH, EQUITY, PROFIT = setAccountVars()
        own_stock = len(positions) > 0

        prediction = stock_predictor.load_data_train_and_predict(TICKER)[0]
        current_close = getAlpacaQuote(TICKER).bp
        print(f'Prediction: {prediction}, Current Close: {current_close}')
        if (prediction > current_close) and not own_stock:
            buyPosition(TICKER, 1, current_close)
        elif (prediction <= current_close) and own_stock:
            sellPosition(TICKER, 1, current_close, False)
run()











[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

           close   Volume       SMA20       SMA50    stddev    upper_bb  \
3495  593.780029  2835661  586.956647  587.113588  5.859983  598.676612   
3496  592.260010  1960963  587.554147  586.987388  5.754889  599.063925   
3497  592.465027  4403041  588.172400  586.888888  5.573261  599.318921   
3498  592.289978  4513509  588.709900  586.827688  5.416110  599.542120   
3499  591.700012  6725109  589.137399  586.760588  5.290131  599.717661   

        lower_bb    upper_kc    lower_kc  squeeze_pro  ...    bb_lower  \
3495  575.236681  595.746621  578.166673            0  ...  575.236681   
3496  576.044370  596.186480  578.921814            0  ...  576.044370   
3497  577.025879  596.532291  579.812509            0  ...  577.025879   
3498  577.877679  596.834065  580.585734            0  ...  577.877679   
3499  578.557138  597.072596  581.202203            0  ...  578.557138   

      squeeze  conversion_line   base_line  leading_span_a  leading_span_b  \
3495        0       592.40

95/96 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0305
Epoch 1: val_loss improved from inf to 0.00044, saving model to training/cp-0001.weights.h5
96/96 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - loss: 0.0301 - val_loss: 4.4382e-04
Epoch 2/3
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 3.0312e-04
Epoch 2: val_loss did not improve from 0.00044
96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 61ms/step - loss: 3.0304e-04 - val_loss: 4.6866e-04
Epoch 3/3
95/96 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 2.4160e-04
Epoch 3: val_loss improved from 0.00044 to 0.00043, saving model to training/cp-0003.weights.h5
96/96 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - loss: 2.4182e-04 - val_loss: 4.3209e-04
Training MSE: 0.00024091903469525278, Testing MSE: 0.0004320915031712502
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step
prediction tail: [[586.3952 ]
 [585.305  ]
 [585.98615]
 [581.6697 ]
 [583.53644]], test tail: [[580.3449707 ]
 [580.51000977]
 [576.02001953]
 [576.98999023]
 [578.22998047]]
Prediction 1 days from now: [583.53644]
E